In [84]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn.metrics import confusion_matrix
import math

In [85]:
FreeSkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/FreeSkel(64).npy")
FlySkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/FlySkel(64).npy")
BackSkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/BackSkel(64).npy")
BreastSkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/BreastSkel(64).npy")
UnderwaterSkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/UnderwaterSkel(64).npy")
DiveSkel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/DiveSkel(64).npy")

FreeLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/FreeLabel(64).npy")
FlyLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/FlyLabel(64).npy")
BackLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/BackLabel(64).npy")
BreastLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/BreastLabel(64).npy")
UnderwaterLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/UnderwaterLabel(64).npy")
DiveLabel = np.load("C:/Users/jonso/OneDrive/Desktop/yolov7-pose-estimation/DiveLabel(64).npy")

x_train = np.concatenate((FreeSkel, FlySkel, BackSkel, BreastSkel, UnderwaterSkel, DiveSkel, UnderwaterSkel))
y_train = np.concatenate((FreeLabel, FlyLabel, BackLabel, BreastLabel, UnderwaterLabel, DiveLabel, UnderwaterLabel))
print(x_train.shape)
print(y_train.shape)

(1858, 64, 17, 2)
(1858, 1)


In [86]:
"""
# Assuming your array is named 'data'
# Shape of data: (576, 128, 17, 2)

# Compute the sum of coordinates across the last axis (2)
sum_coordinates = np.sum(x_train, axis=-1)

# Find the indices where sum of coordinates is not equal to 0
nonzero_indices = np.any(sum_coordinates != 0, axis=(-1, -2))

# Filter the data array using the nonzero indices
preprocessed_data = x_train[nonzero_indices]

# Shape of preprocessed_data: (num_groups, 128, 17, 2)

print(preprocessed_data.shape)
"""

"\n# Assuming your array is named 'data'\n# Shape of data: (576, 128, 17, 2)\n\n# Compute the sum of coordinates across the last axis (2)\nsum_coordinates = np.sum(x_train, axis=-1)\n\n# Find the indices where sum of coordinates is not equal to 0\nnonzero_indices = np.any(sum_coordinates != 0, axis=(-1, -2))\n\n# Filter the data array using the nonzero indices\npreprocessed_data = x_train[nonzero_indices]\n\n# Shape of preprocessed_data: (num_groups, 128, 17, 2)\n\nprint(preprocessed_data.shape)\n"

In [87]:
# Generate the same permutation indices
permutation_indices = np.random.permutation(len(x_train))

# Shuffle both arrays using the same permutation indices
x_train = x_train[permutation_indices]
y_train = y_train[permutation_indices]

print(x_train.shape)
print(y_train.shape)
#print(x_train)
#print(y_train)

(1858, 64, 17, 2)
(1858, 1)


In [88]:
num_classes = 6
input_shape = (64, 17, 2)

y_train = keras.utils.to_categorical(y_train, num_classes)

def build_model3(pretrained=None):
    model = keras.Sequential([
        keras.Input(shape=input_shape),
        layers.Conv2D(64, (2,2), activation = 'relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (2,2), activation = 'relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (2,2), activation = 'relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(), layers.Dropout(0.5),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ])

    if pretrained:
        model.load_weights(pretrained)
    return model
build_model3()

In [89]:
print(y_train.shape)
print(x_train.shape)

(1858, 6)
(1858, 64, 17, 2)


In [95]:
def train(model, sp, epochs=10):
    batch_size = 2
    #model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    #model.compile(loss=SparseCategoricalCrossEntropy(from_logits=True))
    model.fit(
        x_train, 
        y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_split = 0.2)
    #model.evaluate(x_test, y2_test, verbose=1)
    model.save(sp)
    print("-=- Model Saved -=-")
    #predicted_labels=np.argmax(model.predict(x_test), axis=1)
    #confusion_mat = confusion_matrix(y_test, predicted_labels)
    #print(confusion_mat)
    #print(f'Model Accuracy based on testing data: {(np.trace(confusion_mat))/np.sum(confusion_mat)}')

In [96]:
train(build_model3(), "testModel4", epochs=75)


Epoch 1/75
743/743 [==============================] - 7s 8ms/step - loss: 1.7577 - accuracy: 0.3022 - val_loss: 1.5446 - val_accuracy: 0.2608
Epoch 2/75
743/743 [==============================] - 6s 8ms/step - loss: 1.5353 - accuracy: 0.3297 - val_loss: 1.6414 - val_accuracy: 0.3522
Epoch 3/75
743/743 [==============================] - 6s 8ms/step - loss: 1.4039 - accuracy: 0.3950 - val_loss: 1.7837 - val_accuracy: 0.3978
Epoch 4/75
743/743 [==============================] - 6s 8ms/step - loss: 1.3636 - accuracy: 0.4112 - val_loss: 1.9135 - val_accuracy: 0.2688
Epoch 5/75
743/743 [==============================] - 6s 8ms/step - loss: 1.2977 - accuracy: 0.4502 - val_loss: 1.6115 - val_accuracy: 0.4113
Epoch 6/75
743/743 [==============================] - 6s 8ms/step - loss: 1.1928 - accuracy: 0.5087 - val_loss: 1.8911 - val_accuracy: 0.4543
Epoch 7/75
743/743 [==============================] - 6s 8ms/step - loss: 1.0364 - accuracy: 0.5458 - val_loss: 3.6641 - val_accuracy: 0.4220
Epoch 

INFO:tensorflow:Assets written to: testModel4\assets


INFO:tensorflow:Assets written to: testModel4\assets


-=- Model Saved -=-


In [92]:
testModel = tf.keras.models.load_model("testModel2")
testModel.summary
#testModel.predict(FreeSkel)
FL = keras.utils.to_categorical(FlyLabel, num_classes)
testModel.evaluate(FlySkel, FL, verbose=1)
FL = keras.utils.to_categorical(FreeLabel, num_classes)
testModel.evaluate(FreeSkel, FL, verbose=1)
FL = keras.utils.to_categorical(BackLabel, num_classes)
testModel.evaluate(BackSkel, FL, verbose=1)
FL = keras.utils.to_categorical(BreastLabel, num_classes)
testModel.evaluate(BreastSkel, FL, verbose=1)
FL = keras.utils.to_categorical(DiveLabel, num_classes)
testModel.evaluate(DiveSkel, FL, verbose=1)
FL = keras.utils.to_categorical(UnderwaterLabel, num_classes)
testModel.evaluate(UnderwaterSkel, FL, verbose=1)

2/2 [==============================] - 0s 10ms/step - loss: 1.3125 - accuracy: 0.7931


[1.3125228881835938, 0.7931034564971924]